To run this project, `ollama` must be installed. This is a package that allows for LLM usage, easily. 

The `ollama` server can be run using `ollama serve`, for testing purposes. 

In [42]:
import ollama
# Check if models are installed or not.
try:
    print("Installed models on local ollama instance:")
    # print(ollama.list())
    for modelName in ollama.list()["models"]:
        print(modelName["name"])
except:
    raise Exception("ollama server is not online. \nUse ollama serve to run the ollama daemon.")

Installed models on local ollama instance:
phi3:mini


Papers generated use any notes included in the `NOTEFILE`. 

In [43]:
"""Check defaults and make sure there are no exceptions. """

MODELNAME = "phi3:mini"

NOTEFILE = "notes.txt"

try:
    with open(NOTEFILE, "r") as my_file:
        NOTES = "".join(my_file.readlines())
except OSError:
    raise Exception(f"Could not find {NOTEFILE}.")


QUESTIONNUMBER = 5

if QUESTIONNUMBER > 5 or QUESTIONNUMBER < 2:
    raise Exception("Question amount not suitable.")

TOTALMARKS = 100

if TOTALMARKS // QUESTIONNUMBER < 1:
    raise Exception("Number of marks is too low.")

QUESTIONTYPE = "mcq"
""" This can differ between short, long and mcq."""

# try:
#     with open("exampleQuestions.txt", "r") as QUESTIONSFILE:
#         EXAMPLEQUESTIONS = "".join(QUESTIONSFILE.readlines())
# except OSError:
#     raise Exception("Question file does not exist.")

""" If true, then questions can be divided into a,b,c or i, ii, iii.
    Question subdivision support can be added later. 
"""
# if QUESTIONNUMBER > 10:
#     SUBDIVISIONS = True
# else:
#     SUBDIVISIONS = False

' If true, then questions can be divided into a,b,c or i, ii, iii.\n    Question subdivision support can be added later. \n'

In [50]:
class QuestionType:
    """This class houses the different types of questions, and the prompts, that can be created by the program for each paper."""
    names = []

    def __init__(self, name: str, systemPrompt: str, examplePrompt: str = None) -> None:
        self.name = name
        self.names.append(self.name)
        self.systemPrompt = systemPrompt
        self.examplePrompt = examplePrompt


short = QuestionType(
    "short", "You must generate short questions that are clear to understand.")
long = QuestionType(
    "long", "You must generate long answer questions. Examples include comparing and contrasting two concepts.")
mcq = QuestionType(
    "mcq", "You must generate short multiple choice questions. Do not return the answers or options.")

if QUESTIONTYPE not in QuestionType.names:
    raise Exception("Invalid question type.")

In [45]:
""" The system prompt should be designed to use few-shot learning in order to generate the most accurate questions for the source material. """


SYSTEMPROMPT = f"Your job is to generate {QUESTIONNUMBER} questions, in a list. Each question should be written as a string, and should be separated by one new line character \\n each. Questions should not be numbered. \n"

if QUESTIONTYPE == "short":
    SYSTEMPROMPT += "You must generate short questions that are clear to understand."
elif QUESTIONTYPE == "long":
    SYSTEMPROMPT += "You must generate long answer questions. Examples include comparing and contrasting two concepts."
elif QUESTIONTYPE == "mcq":
    MCQOPTIONS = 4
    SYSTEMPROMPT += f"You must generate short multiple choice questions. Do not return the answers or options."
else:
    raise Exception("Question type is invalid.")

"""Adding notes specified by user."""
# if EXAMPLEQUESTIONS != "":
#     SYSTEMPROMPT += f"You must generate questions with a similar structure and form to the questions below: \n {EXAMPLEQUESTIONS}"
SYSTEMPROMPT += f"\nPlease generate the questions from these notes: \n {NOTES} \n"
print(SYSTEMPROMPT)

Your job is to generate 5 questions, in a list. Each question should be written as a string, and should be separated by one new line character \n each. Questions should not be numbered. Each question will be weighted equally, at 20 marks each. 
You must generate short questions that are clear to understand.
Please generate the questions from these notes: 
 A distributed system is a collection of autonomous computers linked by a network, with software to produce an integrated computing infrastructure. 
It is a set of discrete computers that perform a computation together, as if they were a single computing system, as a network of processes, which interact with one another to achieve a goal. These systems are required for processes like resource sharing, computation speed up via parallelism, and fault tolerance and uncertainty management. 
Information is processed data; data that is organised, meaningful + useful. Security is when you prevent unauthorised access to information + services

In [47]:
generation = ollama.generate(model=MODELNAME, prompt=SYSTEMPROMPT)
# print(generation["response"])
generated_questions = [question.replace(
    "\"", "") for question in generation["response"].split("\\n")]
# Filter out the blank spaces.
generated_questions = [question for question in generated_questions if question != ""]

for q in generated_questions:
  print(q)


if len(generated_questions) != QUESTIONNUMBER:
  raise Exception(f"Problem with generated question amount: expected {QUESTIONNUMBER}, got {len(generated_questions)}")
 

# questionChoicesDict = {}
# for question in generated_questions:
#   CHOICEPROMPT = f"Given a question {question}, please generate short {MCQOPTIONS} options, in which {MCQOPTIONS - 1} is incorrect. Output all the options as comma separated strings. Each option should be comma separated. Use the notes {NOTES}. Do not output anything apart from the options.  "
#   generateChoices = ollama.generate(model=MODELNAME, prompt=CHOICEPROMPT)
#   questionChoicesDict[question] = generateChoices["response"]
#   print(generateChoices["response"])

 
W
I
H
C


In [ ]:
""" Add generated questions to a tex file."""
try:
  with open("questions.tex", "w") as my_file:
      
      if QUESTIONTYPE == "mcq":   
        # my_file.write("\\begin{parts}\n")
        for question in generated_questions:  
          my_file.write(
              f"\\question[{TOTALMARKS // QUESTIONNUMBER}] {question} \n")
          my_file.write("\\begin{checkboxes} \n")
          # [my_file.write("\\choice {choices}") for choice in choices]
          my_file.write("\\end{checkboxes} \n")
          # my_file.write(f"\n")
        # my_file.write("\\end{parts}")
except OSError:
   raise Exception("questions.tex does not exist and cannot be written to.")